# **Fine-tuning XLSR-Wav2Vec2 for Multi-Lingual ASR with 🤗 Transformers**

Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.  Soon after the superior performance of Wav2Vec2 was demonstrated on the English ASR dataset LibriSpeech, *Facebook AI* presented XLSR-Wav2Vec2 (click [here](https://arxiv.org/abs/2006.13979)). XLSR stands for *cross-lingual  speech representations* and refers to XLSR-Wav2Vec2's ability to learn speech representations that are useful across multiple languages.

Similar to Wav2Vec2, XLSR-Wav2Vec2 learns powerful speech representations from hundreds of thousands of hours of speech in more than 50 languages of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/xlsr_wav2vec2.png)

The authors show for the first time that massively pretraining an ASR model on cross-lingual unlabeled speech data, followed by language-specific fine-tuning on very little labeled data achieves state-of-the-art results. See Table 1-5 of the official [paper](https://arxiv.org/pdf/2006.13979.pdf).

In this notebook, we will give an in-detail explanation of how XLSR-Wav2Vec2's pretrained checkpoint can be fine-tuned on a low-resource ASR dataset of any language. Note that in this notebook, we will fine-tune XLSR-Wav2Vec2 without making use of a language model. It is much simpler and more efficient to use XLSR-Wav2Vec2 without a language model, but better results can be achieved by including a language model.

For demonstration purposes, we fine-tune the [wav2vec2-large-xlsr-53](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) on the low resource Turkish ASR dataset of [Common Voice](https://huggingface.co/datasets/common_voice) that contains just ~6h of validated training data.

XLSR-Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition.

I highly recommend reading the blog post [Sequence Modeling with CTC (2017)](https://distill.pub/2017/ctc/) very well-written blog post by Awni Hannun.

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `torchaudio` and `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [1]:
%%capture
!pip install --upgrade huggingface_hub
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install soundfile
!pip install evaluate
!pip uninstall accelerate -y
!pip install accelerate -U




---

${}^1$ In the [paper](https://arxiv.org/pdf/2006.13979.pdf), the model was evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). To keep this notebook as general as possible we decided to evaluate the model using WER.

# FONCTIONS

In [ ]:
# ## the goal is to convert the speech array to WAV file in bulk
# audio_file_path = 'DATA\CLEANED\AUDIO_WAV\\'
# def array2WAV(row, df_name, sample_rate = 16000):
#     # Convert NumPy array to AudioSegment object
#     samples = row.get('audio')
#     samples = (samples * (2**15 - 1)).astype(np.int16)
#     audio_segment = AudioSegment(samples.tobytes(), sample_width = 2, frame_rate = sample_rate, channels = 1)

#     # Save the AudioSegment as a WAV file
#     filename = f"{df_name}_{row.get('id')}_WOL.wav"
#     audio_segment.export(os.path.join(audio_file_path, filename), format="wav")

#     # Return the generated file path
#     return filename

In [ ]:
# def ByteWAV2WAV(audio_dict, df_name):
#     try:
#         # Extract 'bytes' from the dictionary
#         audio_bytes = audio_dict.get('audio')['bytes']

#         # Save the bytes as a WAV file
#         filename = f"{df_name}_{audio_dict.get('id')}_WOL.wav"
#         with open(os.path.join(audio_file_path, filename), 'wb') as f:
#             f.write(audio_bytes)

#         # Return the generated file path
#         return filename
#     except Exception as e:
#         print(f"Error processing id {audio_dict.get('id')}: {e}")
#         return None

In [ ]:
# def filename(row, df_name):
#   return f"{df_name}_{row.get('id')}_WOL.wav"

In [ ]:
# def ByteWAV2Array(audio_dict):
#     try:
#         # Extract 'bytes' from the dictionary
#         audio_bytes = audio_dict.get('audio')['bytes']

#         # Load the audio data using librosa
#         audio_data, _ = librosa.load(io.BytesIO(audio_bytes), sr=16000)
#         del audio_bytes
#         # Return the audio data as a NumPy array
#         return audio_data
#     except Exception as e:
#         print(f"Error processing id {audio_dict.get('id')}: {e}")
#         return None

In [ ]:
# def create_id(df, id_max):
#     df['id'] = range(id_max + 1, id_max + len(df) + 1)
#     return df

## Directory handling

In [ ]:
# path = r"C:\Users\maron\OneDrive\02-Documents\03.PROJETS\00.INFORMATIQUE\00.APP\00.SENLAAKK\05.IA\00.WOLOF\SPEECH_TO_TEXT"
# os.chdir(path)

## perrynelson cleaning

### Data loading

In [ ]:
# dataset_waxal = load_dataset("perrynelson/waxal-wolof")
# dataset_waxal_2 = load_dataset("perrynelson/waxal-wolof2")

In [ ]:
# print(f'the keys of the downloaded data are : {dataset_waxal.keys()}')
# print(f'the keys of the downloaded data are : {dataset_waxal_2.keys()}')

In [ ]:
# dataset_waxal_df_train = dataset_waxal['train'].to_pandas()
# dataset_waxal_df_validation = dataset_waxal['validation'].to_pandas()
# dataset_waxal_df_test = dataset_waxal['test'].to_pandas()
# dataset_waxal_2_df_test = dataset_waxal_2['test'].to_pandas()

### Visualization

In [ ]:
# dataset_waxal_df_train.head(3)

In [ ]:
# dataset_waxal_2_df_test.head(3)

In [ ]:
# # Assuming dataset_waxal_df_train, dataset_waxal_df_validation, dataset_waxal_df_test, dataset_waxal_2_df_test are your DataFrames
# dataframes = [dataset_waxal_df_train, dataset_waxal_df_validation, dataset_waxal_df_test, dataset_waxal_2_df_test]

# dataset_waxal_df = pd.concat(dataframes, axis=0, ignore_index=True)
# dataset_waxal_df = create_id(dataset_waxal_df, 0)
# dataset_waxal_df.head(3)

### saving the original data

In [ ]:
#dataset_waxal_df.to_csv("/content/drive/MyDrive/PROJETS/WOLOF_IA/SPEECH_TO_TEXT/DATA/BRUT/CSV/dataset_waxal.csv")

In [ ]:
# # create a new dataframe to store the clean data from perrynelson
# dataset_waxal__clean_df = pd.DataFrame()
# dataset_waxal__clean_df = dataset_waxal_df[["id", "transcription"]]
# dataset_waxal__clean_df['audio'] = dataset_waxal_df.apply(lambda row: ByteWAV2Array(row), axis = 1)
# dataset_waxal__clean_df['filename'] = dataset_waxal_df.apply(lambda row: filename(row, "perrynelson"), axis = 1)

# dataset_waxal__clean_df.head(3)

In [ ]:
# # Save DataFrame
# dataset_waxal__clean_df.to_pickle("DATA/CLEANED/DATA/df_tot_audio_path_transcription.pkl")

# # Load DataFrame
# df = pd.read_pickle("DATA/CLEANED/DATA/df_tot_audio_path_transcription.pkl")


In [ ]:
#dataset_waxal__clean_df.to_pickle("/content/drive/MyDrive/PROJETS/WOLOF_IA/SPEECH_TO_TEXT/DATA/CLEANED/DATA/dataset_waxal__clean_df.pkl")

## Isma cleaning

### Data loading

In [ ]:
# dataset_alffa = load_dataset("Isma/alffa_wolof")

In [ ]:
# print(f'the keys of the downloaded data are : {dataset_alffa.keys()}')

In [ ]:
# dataset_alffa_df_train = dataset_alffa['train'].to_pandas()
# dataset_alffa_df_dev = dataset_alffa['dev'].to_pandas()

In [ ]:
# dataset_alffa_df_train.head(3)

In [ ]:
# dataset_alffa_df_dev.head(3)

In [ ]:
# print(type(dataset_alffa_df_train['audio'][0]))

### Grouping all the data

In [ ]:
# # Assuming dataset_waxal_df_train, dataset_waxal_df_validation, dataset_waxal_df_test, dataset_waxal_2_df_test are your DataFrames
# dataframes = [dataset_alffa_df_train, dataset_alffa_df_dev]

# dataset_alffa_df = pd.concat(dataframes, axis=0, ignore_index=True)
# dataset_alffa_df = create_id(dataset_alffa_df, len(dataset_waxal_df))
# dataset_alffa_df.head(3)

In [ ]:
#dataset_alffa_df.to_pickle("/content/drive/MyDrive/PROJETS/WOLOF_IA/SPEECH_TO_TEXT/DATA/BRUT/CSV/dataset_alffa_df.pkl")

### Sequence to WAV conversion

In [ ]:
# # create a new dataframe to store the clean data from perrynelson
# dataset_alffa_clean_df = pd.DataFrame()
# dataset_alffa_clean_df = dataset_alffa_df[["id", "transcription", "audio"]]
# dataset_alffa_clean_df['filename'] = dataset_alffa_df.apply(lambda row: filename(row, "isma"), axis = 1)

# dataset_alffa_clean_df.head(3)

In [ ]:
#dataset_alffa_clean_df.to_pickle("/content/drive/MyDrive/PROJETS/WOLOF_IA/SPEECH_TO_TEXT/DATA/CLEANED/DATA/dataset_alffa_clean_df.pkl")

## serge-wilson cleaning

### Data loading

In [ ]:
# dataset_serge = load_dataset("serge-wilson/wolof_speech_transcription")

In [ ]:
# print(f'the keys of the downloaded data are : {dataset_serge.keys()}')

In [ ]:
# dataset_serge_df_train = dataset_serge['train'].to_pandas()
# dataset_serge_df_test = dataset_serge['test'].to_pandas()

In [ ]:
# dataset_serge_df_train.head(3)

In [ ]:
# dataset_serge_df_test.head(3)

### Grouping all the data

In [ ]:
# # Assuming dataset_waxal_df_train, dataset_waxal_df_validation, dataset_waxal_df_test, dataset_waxal_2_df_test are your DataFrames
# dataframes = [dataset_serge_df_train, dataset_serge_df_test]

# dataset_serge_df = pd.concat(dataframes, axis = 0, ignore_index=  True)
# dataset_serge_df = create_id(dataset_serge_df, 18571)
# dataset_serge_df.head(3)

In [ ]:
#dataset_serge_df.to_pickle("/content/drive/MyDrive/PROJETS/WOLOF_IA/SPEECH_TO_TEXT/DATA/BRUT/CSV/dataset_serge_df.pkl")

### Bytes to WAV conversion

In [ ]:
# # create a new dataframe to store the clean data from perrynelson
# dataset_serge_clean_df = pd.DataFrame()
# dataset_serge_clean_df['id'] = dataset_serge_df["id"].copy()
# dataset_serge_clean_df["transcription"] = dataset_serge_df["sentence"].copy()
# dataset_serge_clean_df["audio"] = dataset_serge_df.apply(lambda row: ByteWAV2Array(row), axis = 1)
# dataset_serge_clean_df['filename'] = dataset_serge_df.apply(lambda row: filename(row, "serge-wilson"), axis = 1)

# dataset_serge_clean_df.head(3)

In [ ]:
#dataset_serge_clean_df.to_pickle("/content/drive/MyDrive/PROJETS/WOLOF_IA/SPEECH_TO_TEXT/DATA/CLEANED/DATA/dataset_serge_clean_df.pkl")

## Final cleaned data

In [ ]:
# #dataset_waxal__clean_df = pd.read_pickle("/content/drive/MyDrive/PROJETS/WOLOF_IA/SPEECH_TO_TEXT/DATA/CLEANED/DATA/dataset_waxal__clean_df.pkl")
# #dataset_alffa_clean_df = pd.read_pickle("/content/drive/MyDrive/PROJETS/WOLOF_IA/SPEECH_TO_TEXT/DATA/CLEANED/DATA/dataset_alffa_clean_df.pkl")
# #dataset_serge_clean_df = pd.read_pickle("/content/drive/MyDrive/PROJETS/WOLOF_IA/SPEECH_TO_TEXT/DATA/CLEANED/DATA/dataset_serge_clean_df.csv")

# dataframes = [dataset_waxal__clean_df, dataset_alffa_clean_df, dataset_serge_clean_df]

# dataset_tot_cleaned_df = pd.concat(dataframes, axis=0, ignore_index=True)
# dataset_tot_cleaned_df.head(3)

In [ ]:
# dataset_tot_cleaned_df.to_csv("/content/drive/MyDrive/PROJETS/WOLOF_IA/SPEECH_TO_TEXT/DATA/CLEANED/DATA/dataset_tot_cleaned_df.csv")

# MODEL DEVELOPPEMENT

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text.

In 🤗 Transformers, the XLSR-Wav2Vec2 model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).

Let's start by creating the tokenizer responsible for decoding the model's predictions.

### Create Wav2Vec2CTCTokenizer

The [pretrained Wav2Vec2 checkpoint]( ) maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned XLSR-Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *'BERT'* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on XLSR-Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Common Voice and define a vocabulary based on the dataset's transcriptions.

First, let's go to [Common Voice](https://commonvoice.mozilla.org/en/datasets) and pick a language to fine-tune XLSR-Wav2Vec2 on. For this notebook, we will use Turkish.

For each language-specific dataset, you can find a language code corresponding to your chosen language. On [Common Voice](https://commonvoice.mozilla.org/en/datasets), look for the field "Version". The language code then corresponds to the prefix before the underscore. For Turkish, *e.g.* the language code is `"tr"`.

Great, now we can use 🤗 Datasets' simple API to download the data. The dataset name will be `"common_voice"`, the config name corresponds to the language code - `"tr"` in our case.

Common Voice has many different splits including `invalidated`, which refers to data that was not rated as "clean enough" to be considered useful. In this notebook, we will only make use of the splits `"train"`, `"validation"` and `"test"`.

Because the Turkish dataset is so small, we will merge both the validation and training data into a training dataset and simply use the test data for validation.

In [2]:
from datasets import load_dataset, load_metric, Audio
import gc
import numpy as np
import pandas as pd
import wave
import os
import io
import librosa

In [5]:
os.chdir(r"C:\Users\maron\OneDrive\02-Documents\03.PROJETS\00.INFORMATIQUE\02.AI\WOLOF\SPEECH_TO_TEXT")

In [6]:
df_alffa = pd.read_pickle(r"DATA\CLEANED\alffa\alffa_tot.pkl")
df_alffa.head(1)

,Unnamed: 0,id,transcription,length,filename,audio
0,0,1,jén fa nga ko jàppe soo ko fa sange mu rëcc,5.968,isma_1_WOL.wav,"[0.0, 0.0002746582, -0.0009460449, 0.031341553..."


In [89]:
number_of_rows = 100
df_alffa = df_alffa.head(number_of_rows)

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [90]:


# Assuming df_audio_path is a DataFrame and you want to display a random sample
# Make sure df_audio_path has the necessary columns

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    # Use iloc to select rows by integer index
    df_sample = dataset.iloc[picks]
    display(HTML(df_sample.to_html()))

In [91]:
show_random_elements(df_alffa, num_examples=1)

,Unnamed: 0,id,transcription,length,filename,audio
2,2,3,dañu ko logal moo tax déggatuloo ko,5.608,isma_3_WOL.wav,"[6.1035156e-05, -0.00015258789, 0.00033569336, -0.005004883, -0.005554199, -0.002319336, -0.002532959, -0.004486084, -0.0056152344, -0.0067749023, -0.0059814453, -0.005340576, -0.0052490234, -0.004058838, -0.007507324, -0.006866455, -0.006591797, -0.0055236816, -0.0026855469, -0.0020446777, -0.001953125, -0.0037231445, -0.0024414062, -0.0024414062, -0.002532959, -0.0022583008, -0.0034484863, -0.0024719238, -0.004486084, -0.006439209, -0.0060424805, -0.0058288574, -0.004852295, -0.0038146973, -0.0030822754, -0.0025634766, -0.0008239746, -0.0007324219, -0.0026245117, -0.0038146973, -0.0071411133, -0.006591797, -0.005584717, -0.0057373047, -0.0029907227, -0.002105713, -0.0015258789, -0.0020446777, -0.0018920898, -0.002166748, -0.0038757324, -0.003326416, -0.0025634766, -0.002166748, -0.00039672852, 0.0015258789, 0.0008544922, 0.0004272461, -0.00061035156, -0.0032348633, -0.003540039, -0.0037841797, -0.0050354004, -0.003753662, -0.0020751953, -0.0032348633, -0.0027160645, -0.0017700195, -0.0016479492, -0.00091552734, -0.000579834, -0.0023498535, -0.0025634766, -0.001373291, -0.00064086914, 0.00091552734, 0.00012207031, -0.0028381348, -0.005126953, -0.005706787, -0.0050354004, -0.0046081543, -0.0047302246, -0.002960205, -0.0010986328, -0.0017700195, -0.0019836426, -0.0020141602, -0.00289917, -0.002380371, -0.004180908, -0.00491333, -0.0050964355, -0.003112793, -0.0026245117, -0.004425049, -0.0028686523, -0.0026245117, -0.002166748, -0.002960205, ...]"


Alright! The transcriptions look fairly clean. Having translated the transcribed sentences, it seems that the language corresponds more to written-out text than noisy dialogue. This makes sense considering that [Common Voice](https://huggingface.co/datasets/common_voice) is a crowd-sourced read speech corpus.

We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [92]:
import re

chars_to_ignore_regex = r'[,?\.\!\-\;:"“%\‘”�\r\n*/\(\)0-9]'

def remove_special_characters(sentence):
    return re.sub(chars_to_ignore_regex, '', sentence).lower() + " "

# Assuming "transcription" is the name of the column you want to process
df_alffa["transcription"] = df_alffa["transcription"].apply(remove_special_characters)

In [93]:
show_random_elements(df_alffa, num_examples=1)

,Unnamed: 0,id,transcription,length,filename,audio
0,0,1,jén fa nga ko jàppe soo ko fa sange mu rëcc,5.968,isma_1_WOL.wav,"[0.0, 0.0002746582, -0.0009460449, 0.031341553, 0.03933716, 0.031677246, 0.034454346, 0.03643799, 0.0345459, 0.032836914, 0.041656494, 0.042755127, 0.04034424, 0.036590576, 0.034851074, 0.0435791, 0.036499023, 0.02947998, 0.03527832, 0.039031982, 0.04525757, 0.05117798, 0.043701172, 0.035095215, 0.037750244, 0.035827637, 0.033691406, 0.037506104, 0.035217285, 0.035705566, 0.042816162, 0.04586792, 0.05316162, 0.043426514, 0.04248047, 0.041046143, 0.031799316, 0.03652954, 0.040161133, 0.04196167, 0.04559326, 0.048950195, 0.05050659, 0.052001953, 0.05050659, 0.05130005, 0.04989624, 0.049316406, 0.052612305, 0.051605225, 0.051452637, 0.053222656, 0.04748535, 0.049865723, 0.05508423, 0.050964355, 0.052947998, 0.0552063, 0.057250977, 0.059051514, 0.061584473, 0.06439209, 0.06561279, 0.06439209, 0.055114746, 0.05593872, 0.056732178, 0.051757812, 0.056030273, 0.057434082, 0.05441284, 0.051452637, 0.045196533, 0.05117798, 0.056640625, 0.05493164, 0.052093506, 0.052459717, 0.052856445, 0.053222656, 0.05117798, 0.053619385, 0.0541687, 0.052612305, 0.052581787, 0.052734375, 0.06604004, 0.060302734, 0.046875, 0.042999268, 0.053253174, 0.063934326, 0.05883789, 0.048797607, 0.046325684, 0.058288574, 0.0708313, 0.060424805, 0.04736328, 0.04397583, ...]"


Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.

In CTC, it is common to classify speech chunks into letters, so we will do the same here.
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars.
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [94]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df_alffa[["transcription", "filename"]],  # Features (input data)
    df_alffa["audio"],     # Labels (target variable)
    test_size=0.2,              # Proportion of the dataset to include in the test split
    random_state=42             # Seed for reproducibility
)

# Create training DataFrame
df_audio_path_train = pd.DataFrame({"sentence": X_train['transcription'],"path": X_train['filename'] , "audio": y_train})

# Create testing DataFrame
df_audio_path_test = pd.DataFrame({"sentence": X_test['transcription'],"path": X_test['filename'], "audio": y_test})

In [95]:
show_random_elements(df_audio_path_train, num_examples=1)

,sentence,path,audio
1,ngorsi kat masu maa jaar buroom di tataani,isma_2_WOL.wav,"[0.0, -9.1552734e-05, 0.00030517578, -0.008117676, -0.011932373, -0.011169434, -0.011352539, -0.010314941, -0.0121154785, -0.009185791, -0.0069885254, -0.006958008, -0.0061035156, -0.0055236816, -0.0051879883, -0.00592041, -0.0065307617, -0.007293701, -0.0067749023, -0.0065307617, -0.0070495605, -0.0058898926, -0.004699707, -0.004852295, -0.0032043457, -0.001159668, -0.0011291504, -0.000579834, -0.0016784668, -0.0021972656, -0.0018005371, -0.00048828125, 0.000579834, 0.0004272461, 0.0014038086, -6.1035156e-05, 3.0517578e-05, 9.1552734e-05, 0.0006713867, -0.0006713867, -0.00021362305, 0.0024108887, 0.002532959, 0.0035095215, 0.0030517578, 0.0022583008, -0.00064086914, -0.0005187988, -0.0019226074, -0.0030517578, -0.0031433105, -0.0016479492, -0.0007019043, -0.0009765625, -0.0016174316, -0.0028076172, -0.0021972656, -0.0018920898, -0.0014038086, -0.0035705566, -0.0043640137, -0.0028381348, -0.0031738281, -0.0027160645, -0.0018920898, -0.0038146973, -0.0020446777, -0.003753662, -0.0051879883, -0.006713867, -0.008422852, -0.008758545, -0.010467529, -0.0069274902, -0.004547119, -0.005065918, -0.0073547363, -0.008026123, -0.009307861, -0.011016846, -0.01071167, -0.011108398, -0.008514404, -0.006378174, -0.008636475, -0.009796143, -0.010101318, -0.012573242, -0.012878418, -0.011657715, -0.011291504, -0.010986328, -0.010650635, -0.009979248, -0.011169434, -0.010559082, -0.009918213, -0.010345459, -0.008575439, -0.011199951, ...]"


In [96]:
show_random_elements(df_audio_path_test, num_examples=1)

,sentence,path,audio
0,jén fa nga ko jàppe soo ko fa sange mu rëcc,isma_1_WOL.wav,"[0.0, 0.0002746582, -0.0009460449, 0.031341553, 0.03933716, 0.031677246, 0.034454346, 0.03643799, 0.0345459, 0.032836914, 0.041656494, 0.042755127, 0.04034424, 0.036590576, 0.034851074, 0.0435791, 0.036499023, 0.02947998, 0.03527832, 0.039031982, 0.04525757, 0.05117798, 0.043701172, 0.035095215, 0.037750244, 0.035827637, 0.033691406, 0.037506104, 0.035217285, 0.035705566, 0.042816162, 0.04586792, 0.05316162, 0.043426514, 0.04248047, 0.041046143, 0.031799316, 0.03652954, 0.040161133, 0.04196167, 0.04559326, 0.048950195, 0.05050659, 0.052001953, 0.05050659, 0.05130005, 0.04989624, 0.049316406, 0.052612305, 0.051605225, 0.051452637, 0.053222656, 0.04748535, 0.049865723, 0.05508423, 0.050964355, 0.052947998, 0.0552063, 0.057250977, 0.059051514, 0.061584473, 0.06439209, 0.06561279, 0.06439209, 0.055114746, 0.05593872, 0.056732178, 0.051757812, 0.056030273, 0.057434082, 0.05441284, 0.051452637, 0.045196533, 0.05117798, 0.056640625, 0.05493164, 0.052093506, 0.052459717, 0.052856445, 0.053222656, 0.05117798, 0.053619385, 0.0541687, 0.052612305, 0.052581787, 0.052734375, 0.06604004, 0.060302734, 0.046875, 0.042999268, 0.053253174, 0.063934326, 0.05883789, 0.048797607, 0.046325684, 0.058288574, 0.0708313, 0.060424805, 0.04736328, 0.04397583, ...]"


In [97]:
def extract_all_chars(sentence_column):
    all_text = " ".join(sentence_column)
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}

# Assuming "sentence" is the name of the column containing sentences
vocab_train = extract_all_chars(df_audio_path_train["sentence"].tolist())
vocab_test = extract_all_chars(df_audio_path_test["sentence"].tolist())

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [98]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))
vocab_list= list(set(vocab_list))

In [99]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'ñ': 0,
 'r': 1,
 's': 2,
 'f': 3,
 'ë': 4,
 'e': 5,
 'u': 6,
 'é': 7,
 'j': 8,
 'à': 9,
 'p': 10,
 'b': 11,
 'g': 12,
 'k': 13,
 'l': 14,
 'a': 15,
 'c': 16,
 'x': 17,
 'd': 18,
 'i': 19,
 'n': 20,
 't': 21,
 'm': 22,
 ' ': 23,
 'o': 24}

Cool, we see that all letters of the alphabet occur in the dataset (which is not really surprising) and we also extracted the special characters `" "` and `'`. Note that we did not exclude those special characters because:

- The model has to learn to predict when a word is finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.
- From the transcriptions above it seems that words that include an apostrophe, such as `maktouf'un` do exist in Turkish, so I decided to keep the apostrophe in the dataset. This might be a wrong assumption though.

One should always keep in mind that the data-preprocessing is a very important step before training your model. E.g., we don't want our model to differentiate between `a` and `A` just because we forgot to normalize the data. The difference between `a` and `A` does not depend on the "sound" of the letter at all, but more on grammatical rules - *e.g.* use a capitalized letter at the beginning of the sentence. So it is sensible to remove the difference between capitalized and non-capitalized letters so that the model has an easier time learning to transcribe speech.

It is always advantageous to get help from a native speaker of the language you would like to transcribe to verify whether the assumptions you made are sensible, *e.g.* I should have made sure that keeping `'`, but removing other special characters is a sensible choice for Turkish.

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in Common Voice's training set.

Finally, we also add a padding token that corresponds to CTC's "*blank token*". The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [100]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [101]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

27

Cool, now our vocabulary is complete and consists of 53 tokens, which means that the linear layer that we will add on top of the pretrained XLSR-Wav2Vec2 checkpoint will have an output dimension of 53.

Let's now save the vocabulary as a json file.

In [102]:
model_path = r"DATA\MODEL\VOCABS"

import json
with open(model_path +'/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [103]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer(model_path + "/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Next, we will create the feature extractor.

### Create XLSR-Wav2Vec2 Feature Extractor

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus,
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

XLSR-Wav2Vec2 was pretrained on the audio data of [Babel](https://huggingface.co/datasets/librispeech_asr),
[Multilingual LibriSpeech (MLS)](https://ai.facebook.com/blog/a-new-open-data-set-for-multilingual-speech-research/), and [Common Voice](https://huggingface.co/datasets/common_voice). Most of those datasets were sampled at 16kHz, so that Common Voice, sampled at 48kHz, has to be downsampled to 16kHz for training. Therefore, we will have to downsample our fine-tuning data to 16kHz in the following.



A XLSR-Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, XLSR-Wav2Vec2 models should **always** make use of the `attention_mask`.

In [104]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

Great, XLSR-Wav2Vec2's feature extraction pipeline is thereby fully defined!

To make the usage of XLSR-Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [105]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

If one wants to re-use the just created processor and the fine-tuned model of this notebook, one can mount his/her google drive to the notebook and save all relevant files there. To do so, please uncomment the following lines.

We will give the fine-tuned model the name `"wav2vec2-large-xlsr-turkish-demo"`.

In [106]:
processor.save_pretrained(model_path)

[]

Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just the transcriptio. In addition to `sentence`, our datasets include two more column names `path` and `audio`. `path` states the absolute path of the audio file. Let's take a look.


In [107]:
df_audio_path_train["path"].iloc[0]

'isma_2_WOL.wav'

However, `XLSR-Wav2Vec2` expects the input in the format of a 1-dimensional array of 16 kHz. This means that the audio file has to be loaded and resampled.

 Thankfully, `datasets` does this automatically by calling the other column `audio`. Let try it out.

Great, we can see that the audio file has automatically been loaded. This is thanks to the new [`"Audio"` feature](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=audio#datasets.Audio) introduced in `datasets == 4.13.3`, which loads and resamples audio files on-the-fly upon calling.

In the example above we can see that the audio data is loaded with a sampling rate of 48kHZ whereas 16kHz are expected by the model. We can set the audio feature to the correct sampling rate by making use of [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column):

In [108]:
# common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
# common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

Let's take a look at `"audio"` again.

In [109]:
# common_voice_train[0]["audio"]

This seemed to have worked! Let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded.

**Note**: *You can click the following cell a couple of times to listen to different speech samples.*

In [110]:
rand_int = random.randint(0, len(df_audio_path_train)-1)
print(df_audio_path_train["audio"].iloc[rand_int])

[ 0.0000000e+00 -9.1552734e-05  3.0517578e-04 ...  1.1291504e-03
  6.1035156e-04 -2.1972656e-03]


In [111]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(df_audio_path_train)-1)

ipd.Audio(data = df_audio_path_train["audio"].iloc[rand_int], autoplay = True, rate = 16000)

In [112]:
print(df_audio_path_train["audio"].iloc[rand_int])

[ 0.0000000e+00 -9.1552734e-05  3.0517578e-04 ...  1.1291504e-03
  6.1035156e-04 -2.1972656e-03]


Great, it seems like the data is now correctly loaded and resampled.

It can be heard, that the speakers change along with their speaking rate, accent, and background environment, etc. Overall, the recordings sound acceptably clear though, which is to be expected from a crowd-sourced read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [113]:
rand_int = random.randint(0, len(df_audio_path_train)-1)

print("Target text:", df_audio_path_train["sentence"].iloc[rand_int])
print("Input array shape:", df_audio_path_train["audio"].iloc[rand_int].shape)

Target text:  ngorsi kat masu maa jaar buroom di tataani   
Input array shape: (63136,)


In [114]:
a = processor(df_audio_path_train["audio"].iloc[rand_int], sampling_rate = 16000).input_values[0]

Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

Finally, we can leverage `Wav2Vec2Processor` to process the data to the format expected by `Wav2Vec2ForCTC` for training. We will again make use of the `map(...)` function.

First, we load and resample the audio data, simply by calling `batch["audio"]`.
Second, we extract the `input_values` from the loaded audio file. In our case, this includes only normalization, but for other speech models, this step could correspond to extracting, *e.g.* [Log-Mel features](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum).
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

In [115]:
def prepare_dataset(batch):
    audio = batch.get('audio')

    # batched output is "un-batched"
    input_values = processor(audio, sampling_rate=16000).input_values[0]

    # process labels using the `text` argument of the `__call__()` method
    labels = processor(text=batch.get('sentence')).input_ids

    return {'input_values': input_values, 'labels': labels}

df_voice_train = df_audio_path_train.apply(lambda row : prepare_dataset(row), axis = 1)
df_voice_test = df_audio_path_test.apply(lambda row : prepare_dataset(row), axis = 1)

In [116]:
df_voice_train = df_voice_train.reset_index(drop=True)
df_voice_test = df_voice_test.reset_index(drop=True)

# df_voice_train = df_voice_train.stack()
# df_voice_test = df_voice_test.stack()

Let's apply the data preparation function to all examples.

**Note**: Currently `datasets` make use of [`torchaudio`](https://pytorch.org/audio/stable/index.html) and [`librosa`](https://librosa.org/doc/latest/index.html) for audio loading and resampling. If you wish to implement your own costumized data loading/sampling, feel free to just make use of the `"path"` column instead and disregard the `"audio"` column.

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [117]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [118]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [119]:
# wer_metric = load_metric("wer")

In [120]:
from evaluate import load

wer_metric = load("wer")

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [121]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis = -1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `XLSR-Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

Because the dataset is quite small (~6h of training data) and because Common Voice is quite noisy, fine-tuning Facebook's [wav2vec2-large-xlsr-53 checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) seems to require some hyper-parameter tuning. Therefore, I had to play around a bit with different values for dropout, [SpecAugment](https://arxiv.org/abs/1904.08779)'s masking dropout rate, layer dropout, and the learning rate until training seemed to be stable enough.

**Note**: When using this notebook to train XLSR-Wav2Vec2 on another language of Common Voice those hyper-parameter settings might not work very well. Feel free to adapt those depending on your use case.

In [122]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore.
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [123]:
#model.freeze_feature_extractor() # depreciated

In [124]:
model.freeze_feature_encoder()

In addition, let's enable gradient checkpointing to save some memory.

In [125]:
model.gradient_checkpointing_enable()

In a final step, we define all parameters related to training.
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

In [126]:
from transformers import TrainingArguments
model_path = r"DATA\MODEL\RESULTS"
training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-turkish-demo",
  output_dir= model_path + "/wav2vec2-large-xlsr-wolof-test1",
  group_by_length = True,
  per_device_train_batch_size = 2,
  gradient_accumulation_steps = 2,
  evaluation_strategy = "steps",
  num_train_epochs = 1, #30,
  fp16 = False,
  save_steps = 100,
  eval_steps = 100,
  logging_steps = 10,
  learning_rate = 3e-4,
  warmup_steps = 500,
  save_total_limit = 2,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [127]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    data_collator = data_collator,
    args = training_args,
    compute_metrics = compute_metrics,
    train_dataset = df_voice_train,
    eval_dataset = df_voice_test,
    tokenizer = processor.feature_extractor,
)



---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

Training will take multiple hours depending on the GPU allocated to this notebook. While the trained model yields somewhat satisfying results on *Common Voice*'s test data of Turkish, it is by no means an optimally fine-tuned model. The purpose of this notebook is to demonstrate how XLSR-Wav2Vec2's [checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) can be fine-tuned on a low-resource ASR dataset.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

In [128]:
trainer.train()

  0%|          | 0/1 [00:00<?, ?it/s]

c:\Users\maron\anaconda3\Lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
c:\Users\maron\anaconda3\Lib\site-packages\torch\utils\checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'train_runtime': 28.1105, 'train_samples_per_second': 0.071, 'train_steps_per_second': 0.036, 'train_loss': 7.690081596374512, 'epoch': 1.0}


TrainOutput(global_step=1, training_loss=7.690081596374512, metrics={'train_runtime': 28.1105, 'train_samples_per_second': 0.071, 'train_steps_per_second': 0.036, 'train_loss': 7.690081596374512, 'epoch': 1.0})

The training and validation loss go down nicely. The WER starts improving only at a later stage. Because this notebook is just for demonstration purposes, we can stop here.

To fine-tune larger models on larger datasets using CTC loss, one should take a look at the official speech-recognition examples [here](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 🤗.